# Parameters & Sensitivity Analysis

The probabilistic loops Polar can analyze can contain parameters.
Parameters are constants whose value is completely undetermined. Let's look at an example:

In [1]:
!cat loops/random_walk_param.prob

x = 1
while true:
    x = x + 1 {p} x - 1
end


The loop encodes a random walk with an unknown probability `p`. The unknown probability `p` is the parameter.
We can ask the question: "How do small changes in `p` affect the expected value of the loop variable `x`?
To answer this we can, pass `E(x)` in the `goals` option and `p` to the option `sens_diff`.
Polar then computes the derivative of the expected value of `x` with respect to `p`.


In [11]:
!python ../polar.py loops/random_walk_param.prob --goals "E(x)" -sens_diff "p"


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



----------------------
- Sensitivity Result -
----------------------
E(x) = 1; 2*n*p - n + 1
Solution is exact

∂E(x) = 0; 2*n
Solution is exact

Elapsed time: 0.35602426528930664 s


Using the option `sens_diff` Polar first computes the closed-form for the moments passed in `goals` and after that differentiates them with respect to the sensitivity parameter.
There is a second possibility: Polar can directly construct recurrences for the sensitivity of `E(x)` with respect to `p`, so-called "sensitivity recurrences".
To achieve this, instead of the `sens_diff` option we use the `sens` option.
For more details on "sensitivity recurrences" see our paper [Automated Sensitivity Analysis for Probabilistic Loops](https://link.springer.com/chapter/10.1007/978-3-031-47705-8_2).

In [12]:
!python ../polar.py loops/random_walk_param.prob --goals "E(x)" -sens "p"


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



----------------------
- Sensitivity Result -
----------------------
∂E(x) = 0; 2*n
Solution is exact

Elapsed time: 0.3173096179962158 s


We can also perform the same computation in code instead of using the CLI as follows:

In [16]:
from inputparser import Parser
from program import normalize_program
from recurrences import DiffRecBuilder
from recurrences.solver import RecurrenceSolver
from symengine import Symbol

program = Parser().parse_file("loops/random_walk_param.prob")
# Construct normal form so that Polar can analyze it
program = normalize_program(program)

# Use DiffRecBuilder to construct sensitivity recurrences with respect to a given symbol
diff_rec_builder = DiffRecBuilder(program, Symbol("p"))
recurrences = diff_rec_builder.get_recurrences("x")
delta_x = str(diff_rec_builder.delta) + "*x"
solution = RecurrenceSolver(recurrences).get(delta_x)

print(solution)

Piecewise((0, n <= 0), (2*n, True))
